<a href="https://colab.research.google.com/github/Tverdostup1993/Python_for_ds_tasks/blob/main/%D0%A2%D0%B2%D0%B5%D1%80%D0%B4%D0%BE%D1%81%D1%82%D1%83%D0%BF_%D0%9E_%D0%86_%22HW_2_4_%D0%90%D0%BB%D0%B3%D0%BE%D1%80%D0%B8%D1%82%D0%BC%D0%B8_%D0%B1%D1%83%D1%81%D1%82%D0%B8%D0%BD%D0%B3%D1%83_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

В цьому домашньому завданні ми знову працюємо з даними з нашого змагання ["Bank Customer Churn Prediction (DLU Course)"](https://www.kaggle.com/t/7c080c5d8ec64364a93cf4e8f880b6a0).

Тут ми побудуємо рішення задачі класифікації з використанням алгоритмів бустингу: XGBoost та LightGBM, а також використаємо бібліотеку HyperOpt для оптимізації гіперпараметрів.

0. Зчитайте дані `train.csv` в змінну `raw_df` та скористайтесь наведеним кодом нижче аби розділити дані на трнувальні та валідаційні і розділити дані на ознаки з матириці Х та цільову змінну. Назви змінних `train_inputs, train_targets, train_inputs, train_targets` можна змінити на ті, які Вам зручно.

  Наведений скрипт - частина отриманого мною скрипта для обробки даних. Ми тут не викнуємо масштабування та обробку категоріальних змінних, бо хочемо це делегувати алгоритмам, які будемо використовувати. Якщо щось не розумієте в наведених скриптах, рекомендую розібратись: навичка читати код - важлива складова роботи в машинному навчанні.

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from typing import Tuple, Dict, Any


def split_train_val(df: pd.DataFrame, target_col: str, test_size: float = 0.2, random_state: int = 42) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Split the dataframe into training and validation sets.

    Args:
        df (pd.DataFrame): The raw dataframe.
        target_col (str): The target column for stratification.
        test_size (float): The proportion of the dataset to include in the validation split.
        random_state (int): Random state for reproducibility.

    Returns:
        Tuple[pd.DataFrame, pd.DataFrame]: Training and validation dataframes.
    """
    train_df, val_df = train_test_split(df, test_size=test_size, random_state=random_state, stratify=df[target_col])
    return train_df, val_df


def separate_inputs_targets(df: pd.DataFrame, input_cols: list, target_col: str) -> Tuple[pd.DataFrame, pd.Series]:
    """
    Separate inputs and targets from the dataframe.

    Args:
        df (pd.DataFrame): The dataframe.
        input_cols (list): List of input columns.
        target_col (str): Target column.

    Returns:
        Tuple[pd.DataFrame, pd.Series]: DataFrame of inputs and Series of targets.
    """
    inputs = df[input_cols].copy()
    targets = df[target_col].copy()
    return inputs, targets

In [46]:
raw_df = pd.read_csv("train.csv")

TARGET_COL = "Exited"
INPUT_COLS = [
    "CreditScore", "Geography", "Gender", "Age", "Tenure",
    "Balance", "NumOfProducts", "HasCrCard", "IsActiveMember", "EstimatedSalary"
]

train_df, val_df = split_train_val(raw_df, target_col=TARGET_COL)

X_train, y_train = separate_inputs_targets(train_df, INPUT_COLS, TARGET_COL)
X_val, y_val     = separate_inputs_targets(val_df, INPUT_COLS, TARGET_COL)

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)


(12000, 10) (12000,)
(3000, 10) (3000,)


1. В тренувальному та валідаційному наборі перетворіть категоріальні ознаки на тип `category`. Можна це зробити двома способами:
 1. `df[col_name].astype('category')`, як було продемонстровано в лекції
 2. використовуючи метод `pd.Categorical(df[col_name])`

In [5]:
cat_cols = ["Geography", "Gender"]

for col in cat_cols:
    train_df[col] = train_df[col].astype("category")
    val_df[col]   = val_df[col].astype("category")

In [6]:
print(train_df.dtypes)

id                    int64
CustomerId          float64
Surname              object
CreditScore         float64
Geography          category
Gender             category
Age                 float64
Tenure              float64
Balance             float64
NumOfProducts       float64
HasCrCard           float64
IsActiveMember      float64
EstimatedSalary     float64
Exited              float64
dtype: object


In [8]:
for df_ in (train_df, val_df):
    df_["Exited"] = df_["Exited"].astype(int)

INPUT_COLS = [
    "CreditScore", "Geography", "Gender", "Age", "Tenure",
    "Balance", "NumOfProducts", "HasCrCard", "IsActiveMember", "EstimatedSalary"
]
TARGET_COL = "Exited"

X_train, y_train = separate_inputs_targets(train_df, INPUT_COLS, TARGET_COL)
X_val,   y_val   = separate_inputs_targets(val_df,   INPUT_COLS, TARGET_COL)

for c in ["Geography", "Gender"]:
    X_train[c] = X_train[c].astype("category")
    X_val[c]   = X_val[c].astype("category")

In [9]:
print(train_df.dtypes)

id                    int64
CustomerId          float64
Surname              object
CreditScore         float64
Geography          category
Gender             category
Age                 float64
Tenure              float64
Balance             float64
NumOfProducts       float64
HasCrCard           float64
IsActiveMember      float64
EstimatedSalary     float64
Exited                int64
dtype: object


2. Навчіть на отриманих даних модель `XGBoostClassifier`. Параметри алгоритму встановіть на свій розсуд, ми далі будемо їх тюнити. Рекомендую тренувати не дуже складну модель.

  Опис всіх конфігураційних параметрів XGBoostClassifier - тут https://xgboost.readthedocs.io/en/stable/parameter.html#global-config

  **Важливо:** зробіть такі налаштування `XGBoostClassifier` аби він самостійно обробляв незаповнені значення в даних і обробляв категоріальні колонки.

  Можна також, якщо працюєте в Google Colab, увімкнути можливість використання GPU (`Runtime -> Change runtime type -> T4 GPU`) і встановити параметр `device='cuda'` в `XGBoostClassifier` для пришвидшення тренування бустинг моделі.
  
  Після тренування моделі
  1. Виміряйте точність з допомогою AUROC на тренувальному та валідаційному наборах.
  2. Зробіть висновок про отриману модель: вона хороша/погана, чи є high bias/high variance?
  3. Порівняйте якість цієї моделі з тою, що ви отрмали з використанням DecisionTrees раніше. Чи вийшло покращити якість?

In [10]:
from sklearn.metrics import roc_auc_score
from typing import Tuple, List
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier


xgb = XGBClassifier(
    n_estimators=300,
    max_depth=4,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.0,
    min_child_weight=1,
    random_state=42,
    n_jobs=-1,
    tree_method="hist",
    enable_categorical=True,
    eval_metric="auc",
    #device="cuda",
)

xgb.fit(X_train, y_train)

train_proba = xgb.predict_proba(X_train)[:, 1]
val_proba   = xgb.predict_proba(X_val)[:, 1]

train_auc = roc_auc_score(y_train, train_proba)
val_auc   = roc_auc_score(y_val,   val_proba)

print(f"XGBoost AUROC — train: {train_auc:.4f} | val: {val_auc:.4f}")



XGBoost AUROC — train: 0.9568 | val: 0.9353


In [11]:

X_train_enc = pd.get_dummies(X_train, drop_first=True)
X_val_enc   = pd.get_dummies(X_val, drop_first=True)


X_val_enc = X_val_enc.reindex(columns=X_train_enc.columns, fill_value=0)


In [12]:

dt = DecisionTreeClassifier(
    max_depth=5, random_state=42
)
dt.fit(X_train_enc, y_train)
dt_train_auc = roc_auc_score(y_train, dt.predict_proba(X_train_enc)[:, 1])
dt_val_auc   = roc_auc_score(y_val,   dt.predict_proba(X_val_enc)[:, 1])
print(f"DecisionTree AUROC — train: {dt_train_auc:.4f} | val: {dt_val_auc:.4f}")


DecisionTree AUROC — train: 0.9257 | val: 0.9219


Отримана модель XGBoost показує високу якість: AUROC на тренувальних даних становить 0.9568, а на валідаційних — 0.9353, що свідчить про добру здатність до узагальнення та лише незначний розрив між train і val, тобто модель має легкий variance. У порівнянні з Decision Tree (train AUROC = 0.9257, val AUROC = 0.9219), XGBoost дає вищі результати, отже перехід до ансамблевого методу дозволив покращити якість моделі. Але все таки модель потребує доопрацювання

3. Використовуючи бібліотеку `Hyperopt` і приклад пошуку гіперпараметрів для `XGBoostClassifier` з лекції знайдіть оптимальні значення гіперпараметрів `XGBoostClassifier` для нашої задачі. Задайте свою сітку гіперпараметрів виходячи з тих параметрів, які ви б хотіли перебрати. Поставте кількість раундів в підборі гіперпараметрів рівну **20**.

  **Увага!** Для того, аби скористатись hyperopt, нам треба задати функцію `objective`. В ній ми маємо задати loss - це може будь-яка метрика, але бажано використовувтаи ту, яка цільова в вашій задачі. Чим менший лосс - тим ліпша модель на думку hyperopt. Тож, тут нам треба задати loss - негативне значення AUROC. В лекції ми натомість використовували Accuracy.

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення гіперпараметрів
    - створіть в окремій зміній `final_clf` модель `XGBoostClassifier` з найкращими гіперпараметрами
    - навчіть модель `final_clf`
    - оцініть якість моделі `final_clf` на тренувальній і валідаційній вибірках з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи стала вона краще порівняно з попереднім пунктом (2) цього завдання?

In [16]:
import numpy as np
import xgboost as xgb
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from sklearn.metrics import roc_auc_score

space = {
    'n_estimators':     hp.quniform('n_estimators', 200, 800, 50),
    'learning_rate':    hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
    'max_depth':        hp.quniform('max_depth', 3, 7, 1),
    'min_child_weight': hp.quniform('min_child_weight', 1, 6, 1),
    'subsample':        hp.uniform('subsample', 0.6, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 1.0),
    'gamma':            hp.loguniform('gamma', np.log(1e-8), np.log(1.0)),
    'reg_alpha':        hp.loguniform('reg_alpha', np.log(1e-8), np.log(1.0)),
    'reg_lambda':       hp.loguniform('reg_lambda', np.log(0.5), np.log(10.0)),
}

def objective(params):
    params = params.copy()
    params['n_estimators']     = int(params['n_estimators'])
    params['max_depth']        = int(params['max_depth'])
    params['min_child_weight'] = int(params['min_child_weight'])

    clf = xgb.XGBClassifier(
        n_estimators=params['n_estimators'],
        learning_rate=params['learning_rate'],
        max_depth=params['max_depth'],
        min_child_weight=params['min_child_weight'],
        subsample=params['subsample'],
        colsample_bytree=params['colsample_bytree'],
        gamma=params['gamma'],
        reg_alpha=params['reg_alpha'],
        reg_lambda=params['reg_lambda'],
        enable_categorical=True,
        tree_method='hist',
        eval_metric='auc',
        use_label_encoder=False,
        missing=np.nan,
        scale_pos_weight=scale_pos_weight,
        random_state=42,
        n_jobs=-1,
        # device='cuda',
        early_stopping_rounds=50
    )

    clf.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        verbose=False
    )

    val_proba = clf.predict_proba(X_val)[:, 1]
    val_auc = roc_auc_score(y_val, val_proba)
    return {'loss': -val_auc, 'status': STATUS_OK}

trials = Trials()
best = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=20,
    rstate=np.random.default_rng(42)
)

best_cast = {
    'n_estimators':     int(best['n_estimators']),
    'learning_rate':    float(best['learning_rate']),
    'max_depth':        int(best['max_depth']),
    'min_child_weight': int(best['min_child_weight']),
    'subsample':        float(best['subsample']),
    'colsample_bytree': float(best['colsample_bytree']),
    'gamma':            float(best['gamma']),
    'reg_alpha':        float(best['reg_alpha']),
    'reg_lambda':       float(best['reg_lambda']),
}
print("Найкращі гіперпараметри:", best_cast)

final_clf = xgb.XGBClassifier(
    **best_cast,
    enable_categorical=True,
    tree_method='hist',
    eval_metric='auc',
    use_label_encoder=False,
    missing=np.nan,
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    n_jobs=-1,
    device='cuda',
    early_stopping_rounds=50
)

final_clf.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=False
)

train_auc = roc_auc_score(y_train, final_clf.predict_proba(X_train)[:, 1])
val_auc   = roc_auc_score(y_val,   final_clf.predict_proba(X_val)[:, 1])
print(f"final_clf AUROC — train: {train_auc:.4f} | val: {val_auc:.4f}")


  0%|          | 0/20 [00:00<?, ?trial/s, best loss=?]

/usr/local/lib/python3.12/dist-packages/xgboost/callback.py:386: UserWarning: [16:04:42] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  self.starting_round = model.num_boosted_rounds()



 10%|█         | 2/20 [00:02<00:21,  1.20s/trial, best loss: -0.936778585636875]

/usr/local/lib/python3.12/dist-packages/xgboost/callback.py:386: UserWarning: [16:04:44] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  self.starting_round = model.num_boosted_rounds()

/usr/local/lib/python3.12/dist-packages/xgboost/callback.py:386: UserWarning: [16:04:44] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  self.starting_round = model.num_boosted_rounds()



 15%|█▌        | 3/20 [00:03<00:12,  1.32trial/s, best loss: -0.936778585636875]

...

100%|██████████| 20/20 [00:11<00:00,  1.75trial/s, best loss: -0.9368598669318883]

/usr/local/lib/python3.12/dist-packages/xgboost/callback.py:386: UserWarning: [16:04:53] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  self.starting_round = model.num_boosted_rounds()




Найкращі гіперпараметри: {'n_estimators': 400, 'learning_rate': 0.06318442685528432, 'max_depth': 3, 'min_child_weight': 5, 'subsample': 0.950099043659659, 'colsample_bytree': 0.6632855608018366, 'gamma': 0.28814557154898146, 'reg_alpha': 4.307983349647148e-07, 'reg_lambda': 9.639305506171514}


/usr/local/lib/python3.12/dist-packages/xgboost/callback.py:386: UserWarning: [16:04:53] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  self.starting_round = model.num_boosted_rounds()


final_clf AUROC — train: 0.9413 | val: 0.9370


Результат не кращий, валідаційний AUROC майже не змінився (0.9370 проти 0.9353 раніше), зате train знизився з 0.9568 до 0.9413, тож модель стала менше перенавчається.

4. Навчіть на наших даних модель LightGBM. Параметри алгоритму встановіть на свій розсуд, ми далі будемо їх тюнити. Рекомендую тренувати не дуже складну модель.

  Опис всіх конфігураційних параметрів LightGBM - тут https://lightgbm.readthedocs.io/en/latest/Parameters.html

  **Важливо:** зробіть такі налаштування LightGBM аби він самостійно обробляв незаповнені значення в даних і обробляв категоріальні колонки.

  Аби передати категоріальні колонки в LightGBM - необхідно виявити їх індекси і передати в параметрі `cat_feature=cat_feature_indexes`

  Після тренування моделі
  1. Виміряйте точність з допомогою AUROC на тренувальному та валідаційному наборах.
  2. Зробіть висновок про отриману модель: вона хороша/погана, чи є high bias/high variance?
  3. Порівняйте якість цієї моделі з тою, що ви отрмали з використанням XGBoostClassifier раніше. Чи вийшло покращити якість?

In [41]:
import lightgbm as lgb
import numpy as np
from sklearn.metrics import roc_auc_score

cat_cols = [c for c in X_train.columns if str(X_train.dtypes[c]) == "category"]
cat_idx  = [X_train.columns.get_loc(c) for c in cat_cols]

lgbm = lgb.LGBMClassifier(
    objective="binary",
    n_estimators=300,
    learning_rate=0.05,
    num_leaves=25,
    max_depth=-1,
    subsample=0.7,
    colsample_bytree=0.7,
    min_child_samples=20,
    reg_lambda=1.0,
    n_jobs=-1,
    random_state=42,
    device_type="gpu"
)

lgbm.fit(
    X_train, y_train,
    categorical_feature=cat_idx
)

train_auc = roc_auc_score(y_train, lgbm.predict_proba(X_train)[:, 1])
val_auc   = roc_auc_score(y_val,   lgbm.predict_proba(X_val)[:, 1])

print(f"LightGBM AUROC — train: {train_auc:.4f} | val: {val_auc:.4f}")


[LightGBM] [Info] Number of positive: 2442, number of negative: 9558
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 843
[LightGBM] [Info] Number of data points in the train set: 12000, number of used features: 10
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 7 dense feature groups (0.09 MB) transferred to GPU in 0.000644 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.203500 -> initscore=-1.364561
[LightGBM] [Info] Start training from score -1.364561
LightGBM AUROC — train: 0.9734 | val: 0.9344


Великий розрив між train і val (~0.039) свідчить про помітне перенавчання — модель надто добре підлаштувалась під тренувальні дані. Отже, модель добра, але не краща за XGBoost на валідації

5. Використовуючи бібліотеку `Hyperopt` і приклад пошуку гіперпараметрів для `LightGBM` з лекції знайдіть оптимальні значення гіперпараметрів `LightGBM` для нашої задачі. Задайте свою сітку гіперпараметрів виходячи з тих параметрів, які ви б хотіли перебрати. Поставте кількість раундів в підборі гіперпараметрів рівну **10**.

  **Увага!** Для того, аби скористатись hyperopt, нам треба задати функцію `objective`. І тут ми також ставимо loss - негативне значення AUROC, як і при пошуці гіперпараметрів для XGBoost. До речі, можна спробувати написати код так, аби в objective передавати лише модель і не писати схожий код двічі :)

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення гіперпараметрів
    - створіть в окремій зміній `final_lgb_clf` модель `LightGBM` з найкращими гіперпараметрами
    - навчіть модель `final_lgb_clf`
    - оцініть якість моделі `final_lgb_clf` на тренувальній і валідаційній вибірках з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи стала вона краще порівняно з попереднім пунктом (4) цього завдання?

In [44]:
import numpy as np
import lightgbm as lgb
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from sklearn.metrics import roc_auc_score

cat_cols = X_train.select_dtypes(include="category").columns.tolist()
cat_idx  = [X_train.columns.get_loc(c) for c in cat_cols]

pos = float(y_train.sum())
neg = float(len(y_train) - y_train.sum())
scale_pos_weight = (neg / pos) if pos > 0 else 1.0

space = {
    "n_estimators":      hp.quniform("n_estimators", 200, 800, 50),
    "learning_rate":     hp.loguniform("learning_rate", np.log(0.01), np.log(0.2)),
    "num_leaves":        hp.quniform("num_leaves", 16, 64, 4),
    "max_depth":         hp.choice("max_depth", [-1, 4, 6, 8, 10, 12]),
    "min_child_samples": hp.quniform("min_child_samples", 10, 60, 5),
    "subsample":         hp.uniform("subsample", 0.7, 1.0),
    "bagging_freq":      hp.choice("bagging_freq", [0, 1]),
    "colsample_bytree":  hp.uniform("colsample_bytree", 0.7, 1.0),
    "lambda_l1":         hp.loguniform("lambda_l1", np.log(1e-8), np.log(1.0)),
    "lambda_l2":         hp.loguniform("lambda_l2", np.log(0.5), np.log(10.0)),
    "min_split_gain":    hp.loguniform("min_split_gain", np.log(1e-8), np.log(1.0)),
}

def objective(params):
    params = params.copy()
    params["n_estimators"]      = int(params["n_estimators"])
    params["num_leaves"]        = int(params["num_leaves"])
    params["min_child_samples"] = int(params["min_child_samples"])

    clf = lgb.LGBMClassifier(
        objective="binary",
        boosting_type="gbdt",
        metric="auc",
        random_state=42,
        n_jobs=-1,
        n_estimators=params["n_estimators"],
        learning_rate=params["learning_rate"],
        num_leaves=params["num_leaves"],
        max_depth=params["max_depth"],
        min_child_samples=params["min_child_samples"],
        subsample=params["subsample"],
        bagging_freq=params["bagging_freq"],
        colsample_bytree=params["colsample_bytree"],
        lambda_l1=params["lambda_l1"],
        lambda_l2=params["lambda_l2"],
        min_split_gain=params["min_split_gain"],
        scale_pos_weight=scale_pos_weight,
    )

    callbacks = [
        lgb.early_stopping(stopping_rounds=50, verbose=False),
        lgb.log_evaluation(period=0),
    ]

    clf.fit(
        X_train, y_train,
        categorical_feature=cat_idx,
        eval_set=[(X_val, y_val)],
        callbacks=callbacks
    )

    val_auc = roc_auc_score(y_val, clf.predict_proba(X_val)[:, 1])
    return {"loss": -val_auc, "status": STATUS_OK}

trials = Trials()
best = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=10,
    rstate=np.random.default_rng(42)
)
best_cast = {
    "n_estimators":      int(best["n_estimators"]),
    "learning_rate":     float(best["learning_rate"]),
    "num_leaves":        int(best["num_leaves"]),
    "max_depth":         [-1, 4, 6, 8, 10, 12][best["max_depth"]],
    "min_child_samples": int(best["min_child_samples"]),
    "subsample":         float(best["subsample"]),
    "bagging_freq":      [0, 1][best["bagging_freq"]],
    "colsample_bytree":  float(best["colsample_bytree"]),
    "lambda_l1":         float(best["lambda_l1"]),
    "lambda_l2":         float(best["lambda_l2"]),
    "min_split_gain":    float(best["min_split_gain"]),
}
print("Найкращі гіперпараметри:", best_cast)

final_lgb_clf = lgb.LGBMClassifier(
    objective="binary",
    boosting_type="gbdt",
    metric="auc",
    random_state=42,
    n_jobs=-1,
    scale_pos_weight=scale_pos_weight,
    **best_cast
)
final_lgb_clf.fit(
    X_train, y_train,
    categorical_feature=cat_idx,
    eval_set=[(X_val, y_val)],
    callbacks=[lgb.early_stopping(50, verbose=False), lgb.log_evaluation(0)]
)

train_auc = roc_auc_score(y_train, final_lgb_clf.predict_proba(X_train)[:, 1])
val_auc   = roc_auc_score(y_val,   final_lgb_clf.predict_proba(X_val)[:, 1])
print(f"final_lgb_clf AUROC — train: {train_auc:.4f} | val: {val_auc:.4f}")


[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0
[LightGBM] [Warning] lambda_l2 is set=1.319782812839516, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.319782812839516
[LightGBM] [Warning] lambda_l1 is set=7.47272923988995e-05, reg_alpha=0.0 will be ignored. Current value: lambda_l1=7.47272923988995e-05
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0
[LightGBM] [Warning] lambda_l2 is set=1.319782812839516, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.319782812839516
[LightGBM] [Warning] lambda_l1 is set=7.47272923988995e-05, reg_alpha=0.0 will be ignored. Current value: lambda_l1=7.47272923988995e-05
[LightGBM] [Info] Number of positive: 2442, number of negative: 9558
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003302 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory i

Поточний LightGBM після Hyperopt дав найкращий результат: AUROC 0.9470 та 0.9356, що трохи вище за попередній LightGBM 0.9344 (+0.0012) і XGBoost 0.9353. Розрив між train і val невеликий, тож модель виглядає збалансованою

6. Оберіть модель з експериментів в цьому ДЗ і зробіть новий `submission` на Kaggle та додайте код для цього і скріншот скора на публічному лідерборді.
  
  **Напишіть коментар, чому ви обрали саме цю модель?**

  І я вас вітаю - це останнє завдання з цим набором даних 💪 На цьому етапі корисно проаналізувати, які моделі показали себе найкраще і подумати, чому.

Обрала б  LightGBM (після Hyperopt) — він дав найкращу валідацію: AUROC 0.9356 (трохи вище за XGBoost 0.9353) і менший розрив train–val ≈0.011, тож модель більш збалансована й не перенавчається.